In [2]:
import arcpy, os, sys, shutil
from arcpy import env

In [ ]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/network-analyst/build-network.htm
try:
    #Check out Network Analyst license if available. Fail if the Network Analyst license is not available.
    if arcpy.CheckExtension("network") == "Available":
        arcpy.CheckOutExtension("network")
    else:
        raise arcpy.ExecuteError("Network Analyst Extension license is not available.")
    
    #Set environment settings
    env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"
    
    #Set local variables
    inNetworkDataset = "Network_Ped_Bike/Network_ND"
    
    #Build the network dataset
    arcpy.BuildNetwork_na(inNetworkDataset)
    
    #If there are any build errors, they are recorded in a BuildErrors.txt file
    #present in the system temp directory. So copy this file to the directory
    #containing this script
    
    #First get the path to the system temp directory
    tempDir = os.environ.get("TEMP")
    if tempDir:
        shutil.copy2(os.path.join(tempDir,"BuildErrors.txt"),sys.path[0])
    
    print("Script completed successfully.")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))